In [1]:
import os

os.environ['AWS_ENDPOINT_URL'] = 'https://s3.us-west-000.backblazeb2.com'
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''

In [2]:
from tqdm import tqdm
import boto3
import subprocess
from unidecode import unidecode
import IPython.display as ipd
from glob import glob

os.makedirs('part1', exist_ok = True)
os.makedirs('part1-audio', exist_ok = True)

In [3]:
session = boto3.Session()

In [4]:
s3 = session.resource('s3')
bucket = s3.Bucket('imda-singapore')

In [5]:
for f in tqdm(bucket.objects.filter(Prefix='PART1/DATA/CHANNEL0/SCRIPT')):
    filename = os.path.join('part1', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

2034it [00:01, 1458.62it/s]


In [6]:
# for f in tqdm(bucket.objects.filter(Prefix='PART1/DATA/CHANNEL0/WAVE')):
#     filename = os.path.join('part1-audio', os.path.split(f.key)[1])
#     if os.path.exists(filename):
#         continue
                            
#     bucket.download_file(f.key, filename)

In [7]:
# zips = sorted(glob('part1-audio/*.zip'))
# for f in tqdm(zips):
#     os.system(f'unzip -o {f} -d part1-audio > /dev/null')
#     os.remove(f)

In [10]:
import soundfile as sf
from glob import glob

In [11]:
texts = sorted(glob('part1/*'))
len(texts)

2034

In [12]:
filtered = []

for f in tqdm(texts):
    with open(f) as fopen:
        t = fopen.read().split('\n')

    for t_ in t:
        if not len(t_):
            continue
        splitted = t_.split('\t')
        l = unidecode(splitted[0])
        if not len(l):
            continue
        r = unidecode(splitted[1].strip())
        
        speaker = f'SPEAKER{l[1:-4]}'
        session = l[-4]
        filename = os.path.join('part1-audio', speaker, f'SESSION{session}', f'{l}.WAV')
        if not os.path.exists(filename):
            continue
            
        d = {
            'id': l,
            'text': r,
            'filename': filename
        }
        filtered.append(d)
        
len(filtered)

100%|██████████| 2034/2034 [00:04<00:00, 442.03it/s]


755724

In [ ]:
filtered[-1]

In [ ]:
ipd.Audio(filtered[-1]['filename'])

In [13]:
!mkdir part1-mp3

mkdir: cannot create directory ‘part1-mp3’: File exists


In [14]:
import mp

In [17]:
def loop(filtered):
    filtered, _ = filtered
    selected = []
    for f in tqdm(filtered):
        try:
            y, sr = sf.read(f['filename'])
            new_filename = f"part1-mp3/{f['id']}.mp3"
            sf.write(new_filename, y, sr)
            f['new_filename'] = new_filename
            selected.append(f)
        except:
            pass
    return selected

In [25]:
results = mp.multiprocessing(filtered, loop, cores = 20)

 12%|█▏        | 4587/37786 [10:57<37:40, 14.69it/s]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 41%|████▏     | 15602/37786 [36:47<32:32, 11.36it/s]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 73%|███████▎  | 27520/37786 [1:05:05<10:26, 16.39it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--No

In [26]:
len(results)

755724

In [27]:
results[0]

{'id': '000010001',
 'text': 'There were barrels of wine in the huge cellar.',
 'filename': 'part1-audio/SPEAKER0001/SESSION0/000010001.WAV',
 'new_filename': 'part1-mp3/000010001.mp3'}

In [29]:
import json

with open('imda-part1.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 755724/755724 [00:02<00:00, 254721.54it/s]


In [33]:
from huggingface_hub import HfApi
api = HfApi()

In [34]:
api.upload_file(
    path_or_fileobj='imda-part1.jsonl',
    path_in_repo='imda-part1.jsonl',
    repo_id='mesolitica/IMDA-STT',
    repo_type='dataset',
)

imda-part1.jsonl:   0%|          | 0.00/155M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/IMDA-STT/blob/main/imda-part1.jsonl'

In [31]:
# !sudo apt install p7zip-full p7zip-rar -y

In [38]:
!7z -v5g a part1-mp3.7z part1-mp3 > /dev/null

In [41]:
for f in glob('part1-mp3.7z*'):
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id='mesolitica/IMDA-STT',
        repo_type='dataset',
    )

part1-mp3.7z.001


part1-mp3.7z.001:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

part1-mp3.7z.003


part1-mp3.7z.003:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

part1-mp3.7z.002


part1-mp3.7z.002:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

part1-mp3.7z.005


part1-mp3.7z.005:   0%|          | 0.00/298M [00:00<?, ?B/s]

part1-mp3.7z.004


part1-mp3.7z.004:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

In [42]:
total_len = 0
for r in tqdm(results):
    y, sr = sf.read(r['filename'])
    total_len += len(y) / sr

100%|██████████| 755724/755724 [03:37<00:00, 3467.46it/s]


In [43]:
total_len / 60 / 60

1117.9866586978865

In [ ]:
!rm -rf part1-audio